498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9

In [40]:
type Material =
| Rock
| MovingSand
| SettledSand
| Air

type Position = {
    y: int
    x: int
}

type Cave = Map<Position,Material>

module Parser =
    let interpolate (starting,ending) =
        let sorted a b = if a < b then a,b else b,a
        if starting.x = ending.x then
            let (a, b) = sorted starting.y ending.y
            seq {
                for y in [a .. b] do 
                    yield { starting with y = y }
            }
        else
            let (a, b)  = sorted starting.x ending.x
            seq {
                for x in [a .. b] do 
                    yield { starting with x = x }
            }

    let parse (line:string) =
        line.Split("->")
        |> Seq.map ( fun it -> 
            let coords = it.Split(",") |> Array.map int
            { x = coords.[0]; y = coords.[1] } )
        |> Seq.pairwise
        |> Seq.map interpolate
        |> Seq.concat
        |> Seq.map (fun it -> it, Material.Rock)

module Cave =
    let dimensions (cave:Cave) =
        let maxBy chooser = 
            cave 
            |> Map.keys
            |> List.ofSeq
            |> List.map chooser
            |> List.max
        maxBy (fun it -> it.x), maxBy (fun it -> it.y)

    let isFull cave =
        let origin = { x = 500; y = 0 }
        cave 
        |> Map.containsKey origin &&
        match cave |> Map.find origin with
        | Material.SettledSand -> true
        | _ -> false

    let preview (cave:Cave) =
        let width, height = cave |> dimensions
        seq {
            for y in [0 .. height] do yield seq {
                for x in [0 .. width] do
                    let key = { x = x; y = y }
                    if cave |> Map.containsKey key  then 
                        yield
                            match cave |> Map.find key with 
                            | Rock -> "🪨"
                            | MovingSand -> "🟡"
                            | SettledSand -> "🟨"
                            | Air -> "⬛"
                    else
                        yield "⬛"
            }
        }
        |> Seq.map (Seq.skip 450 >> String.concat "")
        |> String.concat "\n"

module Sand =
    let create (cave:Cave) =
        if cave |> Cave.isFull then cave else
        cave
        |> Map.add { x = 500; y = 0 } Material.MovingSand

    let move (sand:Position) (floorOffset: int option) height (cave:Cave) =
        match floorOffset with 
        | Some offset when sand.y = (height + offset) ->
            cave |> Map.change sand (fun it -> match it with Some _ -> Some (Material.SettledSand) | _ -> None) 
        | None when sand.y = height ->
            cave |> Map.remove sand // Sand already at the bottom of the cave
        | _ ->

        let down = { sand with y = sand.y + 1 }
        if cave |> Map.containsKey down then
            let downLeft = { down with x = sand.x - 1 }
            if cave |> Map.containsKey downLeft then
                let downRight = { down with x = sand.x + 1 }
                if cave |> Map.containsKey downRight then
                    cave |> Map.change sand (fun it -> match it with Some _ -> Some (Material.SettledSand) | _ -> None)
                else 
                    cave |> Map.remove sand |> Map.add downRight Material.MovingSand
            else
                cave
                |> Map.remove sand 
                |> Map.add downLeft Material.MovingSand
        else
        cave
        |> Map.remove sand
        |> Map.add down Material.MovingSand

module Simulation =
    let movingSand (cave:Cave) =
        let moving = 
            cave 
            |> Map.toSeq 
            |> Seq.filter (fun (_,mat) -> match mat with Material.MovingSand -> true | _ -> false )

        if moving |> Seq.isEmpty then Seq.empty else
        
        moving
        |> Seq.sortDescending
        |> Seq.map fst

    let settledSand (cave:Cave) =
        cave 
            |> Map.toSeq 
            |> Seq.filter (fun (_,mat) -> match mat with Material.SettledSand -> true | _ -> false )
            |> Seq.map fst

    let tick floorOffset height (cave:Cave) =
        let sand =
            cave 
            |> movingSand
        if sand |> Seq.isEmpty then
            cave
        else
        sand
        |> Seq.fold( fun c s -> c |> Sand.move s floorOffset height) cave
        |> Sand.create

    let rec run ticks floorOffset height cave =
        if ticks = 0 then cave else
        run (ticks - 1) floorOffset height (tick floorOffset height cave)

    let rec solve aTick floorOffset height aCave =
        if aTick > 200 || aCave |> Cave.isFull then aCave, aTick else
        
        let floor, hasFloor =
            match floorOffset with 
            | Some offset -> height + offset, true
            | None -> height, false
        let cave = aCave |> tick floorOffset height
        let sand = cave |> movingSand |> Seq.head
        
        if sand.y >= floor && not hasFloor then 
            cave, aTick
        else
            solve (aTick + 1) floorOffset height cave


let ResolutionFolder = __SOURCE_DIRECTORY__
let cave = 
    File.ReadLines( ResolutionFolder + "/testcase14.txt")
    |> Seq.map Parser.parse 
    |> Seq.concat
    |> Map.ofSeq

let _, height = cave |> Cave.dimensions

let part1, ticks =
    cave 
    //|> Cave.dimensions
    //|> Simulation.run 100 
    |> Simulation.solve 0 None height
    //|> Simulation.movingSand
    //|> Simulation.settledSand |> Seq.length

display ticks

part1 |> Simulation.settledSand |> Seq.length |> display

part1 |> Cave.preview

Stopped due to error


Error: input.fsx (120,12)-(120,30) typecheck error The type 'Position * Material' does not match the type 'Position'
input.fsx (136,48)-(136,49) typecheck error This expression was expected to have type
    'Position'    
but here has type
    'Position * Material'    
input.fsx (153,12)-(153,18) typecheck error Type constraint mismatch. The type 
    'Position * Material'    
is not compatible with type
    'Position'    


In [38]:
let part2, ticks =
    cave 
    |> Simulation.solve 0 (Some 2) height

display ticks

part2 |> Cave.preview

Error: System.ArgumentException: The input sequence was empty. (Parameter 'source')
   at Microsoft.FSharp.Collections.SeqModule.Head[T](IEnumerable`1 source) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 1653
   at FSI_0067.Simulation.solve(Int32 aTick, FSharpOption`1 floorOffset, Int32 height, FSharpMap`2 aCave)
   at <StartupCode$FSI_0071>.$FSI_0071.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)